In [1]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import SQLContext, Row  
from pyspark.sql import functions as F
sc =SparkContext()
spark = SparkSession(sc)
sqlContext=SQLContext(sc)


In [2]:
temps= sc.textFile("C:/Users/quartermaine/Documents/Big Data Analytics Labs/temperature-readings-tiny.csv")
parts = temps.map(lambda l:l.split(";"))
tempReadings = parts.map(lambda p: Row(station=p[0],  date=p[1], year=p[1].split("-")[0],month=p[1].split("-")[1], time=p[2],  Temp=float(p[3]), quality=p[4]))
#Inferring the schema and registering the DataFrame as atable
schemaTempReadings =  sqlContext.createDataFrame(tempReadings)
schemaTempReadings.registerTempTable("tempReadings")

maxTempsYearStation=schemaTempReadings.select(["year","station","Temp"]).filter((schemaTempReadings['year'] <=2014) & (schemaTempReadings["year"]>=1950 ))\
.groupby(['year','station']).agg(F.max('Temp').alias("max_temp")).orderBy('max_temp',ascending=False)

minTempsYearStation=schemaTempReadings.select(["year","station","Temp"]).filter((schemaTempReadings['year'] <=2014) & (schemaTempReadings["year"]>=1950 ))\
.groupby(['year','station']).agg(F.min('Temp').alias("min_temp")).orderBy('min_temp',ascending=False)

In [3]:
maxTempsYearStation.show()
minTempsYearStation.show()

+----+-------+--------+
|year|station|max_temp|
+----+-------+--------+
|2014| 102170|    29.1|
|1960| 102190|    29.0|
|1959| 102190|    28.2|
|1958| 102190|    28.1|
|1956| 102190|    26.0|
|1957| 102190|    25.2|
|1955| 102190|    20.4|
|1961| 102190|    19.0|
|2013| 102170|    10.2|
+----+-------+--------+

+----+-------+--------+
|year|station|min_temp|
+----+-------+--------+
|2013| 102170|   -13.3|
|1957| 102190|   -19.9|
|1959| 102190|   -23.2|
|1961| 102190|   -23.5|
|2014| 102170|   -24.3|
|1955| 102190|   -26.2|
|1958| 102190|   -27.9|
|1960| 102190|   -28.3|
|1956| 102190|   -30.0|
+----+-------+--------+

